In [48]:
def parse_input(input_path='test.txt'):
    with open(input_path, 'r') as f:
        lines = f.read().splitlines()
    
    directions = lines[0]
    directions = [0 if char == 'L' else 1 for char in directions]
    
    # parse out mapping, no need for regex since pattern is simple
    mapping = {}
    for line in lines[2:]:
        mapping[line[0:3]] = (line[7:10], line[12:15])

    return directions, mapping
directions, mapping = parse_input('input.txt')

In [40]:
def traverse_path(directions, mapping):
    n_steps = 0
    start = 'AAA'
    current = 'AAA'
    n_directions = len(directions)

    while current != 'ZZZ':
        # take a step
        direction = directions[(n_steps % n_directions)]
        current = mapping[current][direction]
        n_steps += 1
    return n_steps
n_steps = traverse_path(directions, mapping)

KeyError: 'AAA'

In [36]:
n_steps

6

## Part 2

gotta figure out starting points

In [72]:
def find_starting_points(mapping):
    starting_points = []
    for key in mapping.keys():
        if key[2] == 'A':
            starting_points.append(key)
    return starting_points

starting_points = find_starting_points(mapping)
print('starting_points:', starting_points)

starting_points: ['DNA', 'HNA', 'AAA', 'LMA', 'VGA', 'LLA']


In [ ]:
def traverse_parallel(directions, mapping, starting_points):
    n_steps = 0
    n_directions = len(directions)
    current_positions = starting_points
    current_end = [char[2] for char in starting_points]
    goal = ['Z' for _ in starting_points]

    print(current_end, goal)
    while current_end != goal:
        # print(current_positions)
        direction = directions[(n_steps % n_directions)]
        for i, point in enumerate(current_positions):
            current_positions[i] = mapping[point][direction]
            current_end = [char[2] for char in current_positions]
        n_steps += 1
    print('got to:', current_positions)
    return n_steps
traverse_parallel(directions, mapping, starting_points)


### Brute force is too slow!
- we need to look at individual cycles of when:
- we get from A -> **Z_0** -> .. -> Zn -> .. -> **Z_0**

In [73]:
def simulate_cycle(pos, directions, mapping):
    n_steps, n_directions, first_z = 0, len(directions), None
    cycle = []
    current = pos
    while True:
        while n_steps == 0 or current[2] != 'Z':
            # take a step
            direction = directions[n_steps % n_directions]
            current = mapping[current][direction]
            n_steps += 1
        
        cycle.append(n_steps)
        
        if first_z is None:
            first_z = current
            n_steps = 0
        elif current == first_z:
            # end of cycle
            break
        else:
            direction = directions[n_steps % n_directions]
            current = mapping[current][direction]
            n_steps += 1

    print('cycle for start pos', pos, ':', cycle)
    return cycle

In [74]:
cycles = []
for point in starting_points:
    
    cycles.append(simulate_cycle(point, directions, mapping))

cycle for start pos DNA : [20569, 20569]
cycle for start pos HNA : [18727, 18727]
cycle for start pos AAA : [14429, 14429]
cycle for start pos LMA : [13201, 13201]
cycle for start pos VGA : [18113, 18113]
cycle for start pos LLA : [22411, 22411]


In [76]:
# cycles are same values so the loop is nice and even, no offsets
# need to find the lowest common denominator
from math import gcd


lcm = cycles[0][0]
for cycle in cycles[1:]:
    lcm *= cycle[0] // gcd(lcm, cycle[0])
print('lcm:', lcm)

lcm: 10921547990923
